In [1]:
import pandas as pd
from io import BytesIO
import requests
import bs4
import gspread
from datetime import datetime
import sqlite3
import time

In [2]:
# python-read-sheet@read-sheets-330912.iam.gserviceaccount.com
google_account = gspread.service_account(filename = 'credentials.json')

In [5]:
zhuxian = '1SoQgmDGRmkRXWyp-z9wG-GxAsLAEy4inoGCRjvFPuTc' # 主线任务对话
renwuqipao = '1v2moS-5NOzojAmHrKSWrDjGIV0pNMxApwrr-HuEmFVE' # 任务气泡
sanxuanqipao = '1Jp8a0AIA1IxgddLqE95Kcwh1IY4q5DAXVQm8V1DXa_4' # 三选气泡
renwuming = '1jEog1j6CuBkejZMW-lTfcEFsUe53qZfsEebHJHLnfHs' # 主线任务名
renwuzhangjie = '1fXJC_ebntfN1jht1ejj661yIGfdZbeLSNJVkpTNQBFQ' # 任务章节
renwumingcheng = '1liBCMT0tAuvstxrFI-vB18cs9wLJ4-zRJ4kVS3-LX9c' # 人物名称
renwuduihua = '1Uvcww5ha4-MKSb54-OJ28T59Gbd9U_pTzj_go1BqW1g' # 支线任务对话
zhixianrenwuming = '1hpxlS3JyhImxNIpYrDxxZmRI7a9QyD6pGuJUL_39AA0' # 支线房间任务名
zhixianlinju = '17xl0g8o7n8rw9MM29jDLuUq1tmJ7hDljB8kpzZfoygY' # 支线房间邻居
juqingduihua = '1AxitsKtssepcldDQ0A41SHonsidNUzfL6yc7O9Kf3EU' # 活动剧情对话
huodongqipao = '1flfhwy7S3m8-XqhppGGHCp8Y50TYSPLxuX80rM54Za4' # 活动AI气泡
ui = '1ce6JZQd-my_rdK_jnPCwrnqSsqppNM1X7TBfweqQFXc' # UI文字
youjian = '14OiNT1er_lOYOldhImg3MCFRozPJFRkjy0o5VKJqncU' # 邮件文本
qipao = '1xFHfpBpPqSMrNuwMPTef6-25iLcQVdeVt2GUfVqx0Qw' # 气泡
sanxuanjiaju = '1nUFzvi0iaoCCVG1FTY_NomEgIC8B3WxKC1GceOtFZ8A' # 三选家具名
jiajuming = '1YfAF3jORy3lxlKiTMFFOCwbFXyBmW1FPBaTWFdwdjcY' # 家具名称

In [6]:
def read_spreadsheet(key):
    response = google_account.open_by_key(key)
    if key == zhuxian:
        file_name = '主线任务对话'
    elif key == renwuqipao:
        file_name = '任务气泡'
    elif key == sanxuanqipao:
        file_name = '三选气泡'
    elif key == renwuming:
        file_name = '主线任务名'
    elif key == renwuzhangjie:
        file_name = '任务章节'
    elif key == renwumingcheng:
        file_name = '人物名称'
    elif key == jiajuming:
        file_name = '家具名称'
    elif key == renwuduihua:
        file_name = '支线任务对话'
    elif key == zhixianrenwuming:
        file_name = '支线任务名'
    elif key == zhixianlinju:
        file_name = '支线房间邻居'
    elif key == juqingduihua:
        file_name = '活动剧情对话'
    elif key == huodongqipao:
        file_name = '活动AI气泡'
    elif key == ui:
        file_name = 'UI文字'
    elif key == youjian:
        file_name = '邮件文本'
    elif key == qipao:
        file_name = '气泡'
    elif key == sanxuanjiaju:
        file_name = '三选家具名'
        
    try:
        sheet = response.worksheet("#总表")
        print("总表")
        sheet = sheet.get_all_records()
        df = pd.DataFrame(sheet)
        df.insert(loc=0, column='sheet_name', value='总表')
        df.insert(loc=0, column='file_name', value=str(file_name))
    except:
        df = pd.DataFrame()
        length = len(response.worksheets())
        print("numbers of Sheets: {}".format(length))
        for num in range(length):
            sheet = response.get_worksheet(num)
            sheet_df = pd.DataFrame(sheet.get_all_records())
            if len(sheet_df.columns) > 15:
                sheet_df.insert(loc=0, column='sheet_name', value=str(sheet.title))
                sheet_df.insert(loc=0, column='file_name', value=str(file_name))
                df = df.append(sheet_df)                
#         df = df.drop(df[(df['id']=='long') | (df['id']=='string') | (df['id']=='int')].index) # id == long or string or int
#         df = df.drop(df[df['en']=='英语'].index) # en = 英语 
#         df = df.drop(df[(df['en']=='英语') | (df['en']==' ')].index) # en = 英语 
#         df = df.dropna(subset=['en']) # en 空值
    print("Length: {}".format(len(df)))    
    df = df.reset_index(drop = True)
    return df

def set_columns(df):
    df.columns = df.columns.str.lower().to_list()
    df = df[['file_name', 'sheet_name', 'id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th']]
    df.columns = ['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th']
    return df

def drop_trash(df):
    df = df.drop(df[(df['text_id']=='long') | (df['text_id']=='程序用') | (df['text_id']=='程序用ID') | (df['text_id']=='程序用家具ID') | (df['text_id']=='string') | (df['text_id']=='int')].index)
    df = df.drop(df[df['en'].str.len()==0].index)
    return df

In [7]:
# 主线任务对话
zhuxian_spread = read_spreadsheet(zhuxian)
zhuxian_spread = set_columns(zhuxian_spread)
zhuxian_spread = drop_trash(zhuxian_spread)
print(len(zhuxian_spread))
# time.sleep(60)
zhuxian_spread.head(2)

numbers of Sheets: 14
Length: 9441
9401


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
2,主线任务对话,#译文总表,NaN,程序用,英语,简体中文,日语,德语,法语,繁体中文,意大利语,俄罗斯语,西班牙语,葡萄牙语,韩语,土耳其语,阿拉伯语,印尼语,泰语
3,主线任务对话,#译文总表,NaN,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string


In [8]:
# 任务气泡
time.sleep(60)
renwuqipao_spread = read_spreadsheet(renwuqipao)
renwuqipao_spread = set_columns(renwuqipao_spread)
renwuqipao_spread = drop_trash(renwuqipao_spread)
print(len(renwuqipao_spread))
renwuqipao_spread.head(2)

numbers of Sheets: 14
Length: 6306
6265


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,任务气泡,#译文总表,31254,FALSE,Look at her now! We'll be able to sail in no t...,,見てくれ！今すぐ出港できるぞ！,Schaut sie euch jetzt an! Wir werden im Handum...,Regardez-moi ça ! Il est bientôt prêt à prendr...,看！我們很快就能乘船啟航啦！,Guardala adesso! Potremo salpare in men che no...,Вы только полюбуйтесь этой красавицей! Хоть се...,¡Mira eso! ¡Pronto estaremos listos para zarpar!,Olhe para ela agora! Poderemos de navegar rapi...,봐봐! 머지않아 출항할 수 있겠어!,Şuna bir bak! Yakında denize açılabileceğiz!,ألق نظرة عليه الآن! سنكون قادرين على الإبحار ف...,Lihat dia sekarang! Sebentar lagi kita bisa be...,ดูมันตอนนี้สิ! เราจะออกเรือกันได้ในอีกไม่นานแล้ว!
6,任务气泡,#译文总表,31255,FALSE,"It's good that Jojo is going with you, Alice.",,ジョーイが一緒に行ってくれるなら安心ね、アリス。,"Es ist gut, dass Jojo mit dir kommt, Alice.","C’est bien que Juju t’accompagne, Alice.",幸好JOJO會陪你去呢，艾莉絲。,"È bello che Jojo venga con te, Alice.","Здорово, что тебя будет сопровождать Джо-Джо, ...","Qué bueno que Jojo va a ir contigo, Alice.","É bom que Jojo vá com você, Alice.","조조가 같이 간다니 잘됐구나, 앨리스.",Jojo’nun seninle gelmesi iyi olmuş Alice.,من الجيد أن «جوجو» سيرافقك يا أليس.,"Kamu beruntung Jojo pergi denganmu, Alice.",ดีจังนะที่โจโจจะไปกับเธอด้วย อลิซ


In [9]:
# 三选气泡
time.sleep(60)
sanxuanqipao_spread = read_spreadsheet(sanxuanqipao)
sanxuanqipao_spread = set_columns(sanxuanqipao_spread)
sanxuanqipao_spread = drop_trash(sanxuanqipao_spread)
print(len(sanxuanqipao_spread))
sanxuanqipao_spread.head(2)

numbers of Sheets: 11
Length: 4156
3501


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
622,三选气泡,#译文总表,1010110180020001,FALSE,White is so appealing.,,白はとても魅力的だわ。,Weiß ist so ansprechend.,Le blanc est une super couleur.,白色就很誘人。,Il bianco è proprio accattivante.,Белый — это так трогательно.,El blanco es muy atractivo.,Branco é tão atraente.,흰색은 참 매력 있어.,"Beyaz, çok çekici.",الأبيض جذاب للغاية.,Putih itu sangat menarik.,สีขาวดูน่าดึงดูดใจดีนะ
623,三选气泡,#译文总表,1010510380020001,FALSE,Popsicles bring such a summer vibe!,,アイスは夏の雰囲気を運んできてくれるわね。,Eis am Stiel gibt einem immer so ein Sommergef...,Les glaces à l’eau font vraiment été !,冰棒充滿了夏天的氣息！,I ghiaccioli sanno proprio di estate!,Все эти фрукты словно приближают лето!,¡Las paletas brindan un ambiente de verano!,Picolés trazem um clima tão de verão!,하드가 여름 느낌을 물씬 내 주네!,Dondurmanın getirdiği yaz havasını çok seviyorum!,تضفي المصاصات المُثلَجة جو صيف استرخائي.,Es loli memberikan suasana musim panas!,ไอศกรีมช่วยสร้างบรรยากาศแบบฤดูร้อน!


In [10]:
# 主线任务名
time.sleep(60)
renwuming_spread = read_spreadsheet(renwuming)
renwuming_spread = set_columns(renwuming_spread)
renwuming_spread = drop_trash(renwuming_spread)
print(len(renwuming_spread))
renwuming_spread.head(2)

numbers of Sheets: 14
Length: 1976
1935


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,主线任务名,#译文总表,FALSE,T_GaragePlacechairsin,Place bar stools,,ベッドスツールを設置する,Barhocker platzieren,Placer des tabourets,裝些吧檯椅,Metti degli sgabelli,Поставьте барные стулья,Colocar taburetes del bar,Colocar bancos de bar,높은 걸상 놓기,Bar taburelerini yerleştir,ضع كراسي البار,Tempatkan kursi bar,จัดวางเก้าอี้บาร์
6,主线任务名,#译文总表,FALSE,T_DockCleanyacht,Install propellers,,プロペラを設置する,Schiffsschraube installieren,Installer les hélices,安裝螺旋槳,Installa le eliche,Установить винты,Instalar hélices,Instalar hélices,프로펠러 설치하기,Pervaneleri tak,تركيب المراوح الدافعة,Pasang baling-baling,ติดตั้งใบพัด


In [11]:
# 任务章节
time.sleep(60)
renwuzhangjie_spread = read_spreadsheet(renwuzhangjie)
renwuzhangjie_spread['key'] = ''
renwuzhangjie_spread = set_columns(renwuzhangjie_spread)
renwuzhangjie_spread = drop_trash(renwuzhangjie_spread)
print(len(renwuzhangjie_spread))
renwuzhangjie_spread.head(2)

总表
Length: 55
53


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
2,任务章节,总表,1010100101,,A wind of change is coming...,拯救庄园,変化の風が吹いてきた…,Ein Wind der Veränderungen kommt ...,Le vent du changement se lève...,蛻變之風即將到來...,È in arrivo un vento di cambiamento...,Грядет ветер перемен...,Se aproxima una brisa de cambio...,Ventos de mudança estão chegando...,변화의 바람이 불어옵니다...,Değişim rüzgârı yolda...,رياح التغيير آتية...,Angin perubahan datang...,สายลมแห่งการเปลี่ยนแปลงกำลังมาถึง...
3,任务章节,总表,1010100201,,Change Alan's mind about renovating!,说服艾伦,改装に対するアランの気持ちを変えよう,Stimme Alan bezüglich Renovationen um!,Montrez à Alain que rénover est une bonne chose !,改變阿朗叔對裝修的看法！,Fai cambiare idea ad Alan sulla ristrutturazione!,Измените отношение Алана к реставрации!,¡Cambia la opinión de Alan sobre la renovación!,Faça o Alan mudar de ideia sobre a renovação!,집 단장을 반기지 않는 앨런을 설득하세요!,Alan’ın tadilat hakkındaki fikirlerini değiştir!,تغيير رأي ألن بشأن التجديد!,Ubah pikiran Alan soal renovasi!,เปลี่ยนใจอลันเรื่องการปรับปรุงคฤหาสน์!


In [12]:
# 人物名称
time.sleep(60)
renwumingcheng_spread = read_spreadsheet(renwumingcheng)
renwumingcheng_spread = renwumingcheng_spread.rename(columns={'#key':'key'})
renwumingcheng_spread = set_columns(renwumingcheng_spread)
renwumingcheng_spread = drop_trash(renwumingcheng_spread)
print(len(renwumingcheng_spread))
renwumingcheng_spread.head(2)

总表
Length: 26
23


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
3,人物名称,总表,Alice,Alice,Alice,爱丽丝,アリス,Alice,Alice,艾莉絲,Alice,Алиса,Alice,Alice,앨리스,Alice,أليس,Alice,อลิซ
4,人物名称,总表,Joey,Joey,Joey,乔伊,ジョーイ,Joey,Julien,喬伊,Joey,Джои,Joey,Joey,조이,Joey,جُوي,Joey,โจอี้


In [13]:
# 家具名称
time.sleep(60)
jiajuming_spread = read_spreadsheet(jiajuming)
jiajuming_spread = jiajuming_spread.rename(columns={'#家具名称':'key'})
jiajuming_spread = set_columns(jiajuming_spread)
jiajuming_spread = drop_trash(jiajuming_spread)
print(len(jiajuming_spread))
jiajuming_spread.head(2)

总表
Length: 1144
1143


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
1,家具名称,总表,10101100200200,A1_小路1/3,Rosette,玫瑰花环,ロゼッタ,Rosette,Rosette,玫瑰設計,Rosette,Роза,Rosette,Roseta,장미,Rosette,,,
2,家具名称,总表,10101100200300,A1_小路2/3,Promenade,新式古典,プロムナード,Promenade,Promenade,長廊大道,Passeggiata,Променад,Paseo,Alameda,산책,Kordon,,,


In [14]:
# 支线任务对话
time.sleep(60)
renwuduihua_spread = read_spreadsheet(renwuduihua)
# renwuduihua_spread['key'] = ''
renwuduihua_spread = set_columns(renwuduihua_spread)
renwuduihua_spread = drop_trash(renwuduihua_spread)
print(len(renwuduihua_spread))
renwuduihua_spread.head(2)

numbers of Sheets: 11
Length: 1274
1221


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,支线任务对话,#译文总表,FALSE,R_MichelleRoom1_1,"Guys, I'm low-key freaking out--Darius is comi...",,うそでしょ！やばいやばい！ダリウスが街にやってくるんだって！,"Ich bin fast am durchdrehen, Leute – Darius ko...",Je suis dans tous mes états... Dylan va venir ...,大家，我現在超興奮的——大流士要來我們的小鎮了！！,"Ragazzi, non posso crederci... Darius verrà in...","Народ, я вот прям сейчас в обморок хлопнусь — ...","Chicos, estoy alucinando... ¡Darius viene al p...","Pessoal, estou meio que pirando...Darius está ...","와, 저 기절하기 직전이에요-- 다리우스가 이 마을에 온대요!","Millet, havalara uçuyorum --Darius kasabaya ge...",أنا مذعورة جدًا يا رفاق - «دارا» في طريقه إلى ...,"Kawan-kawan, aku senang sekali--Darius akan ke...",ทุกคน ฉันตื่นเต้นจนสติแทบหลุดแล้ว ดาเรียสกำลัง...
6,支线任务对话,#译文总表,FALSE,T_MichelleBarBuildstepstones_3,Wow! That would be the best thing ever!,,嘘でしょ！めちゃくちゃ嬉しいわ！,Boah! Das wäre das Beste überhaupt!,Ouah ! Ça serait génial !,哇啊！真的嗎？！那也太棒了！,Wow! Sarebbe favoloso!,О-ОХ! Это просто мега-супер!,¡Guau! ¡Eso sería fabuloso!,Uau! Isso seria a melhor coisa de todos os tem...,우와! 그래 주면 더 바랄 게 없겠네요!,Oha! Bu muhteşem olurdu!,رائع! سيكون هذا أفضل شيء على الإطلاق!,Wow! Itu sangat luar biasa!,ว้าว! ยอดเยี่ยมที่สุดเลยค่ะ!


In [15]:
# 支线房间任务名
time.sleep(60)
zhixianrenwuming_spread = read_spreadsheet(zhixianrenwuming)
# zhixianrenwuming_spread['key'] = ''
zhixianrenwuming_spread = set_columns(zhixianrenwuming_spread)
zhixianrenwuming_spread = drop_trash(zhixianrenwuming_spread)
print(len(zhixianrenwuming_spread))
zhixianrenwuming_spread.head(2)

numbers of Sheets: 11
Length: 518
472


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,支线任务名,#译文总表,FALSE,T_EgyptHelpreggie,Help Reggie,,レジ―を助ける,Reggie helfen,Aider Réginald,協助雷吉,Aiuta Reggie,Помогите Реджи,Ayudar a Reggie,Ajudar o Reggie,레지 돕기,Reggie’ye yardım et,مساعدة ريجي,Bantu Reggie,ช่วยเร็จจี้
6,支线任务名,#译文总表,FALSE,T_EgyptRoomDigA,Dig,,掘る,Graben,Creuser,挖掘,Scava,Рыть,Cavar,Escavar,파기,Kaz,حفر,Gali,ขุด


In [16]:
# 支线房间邻居
time.sleep(60)
zhixianlinju_spread = read_spreadsheet(zhixianlinju)
zhixianlinju_spread['key'] = ''
zhixianlinju_spread = set_columns(zhixianlinju_spread)
zhixianlinju_spread = drop_trash(zhixianlinju_spread)
print(len(zhixianlinju_spread))
zhixianlinju_spread.head(2)

总表
Length: 21
18


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
3,支线房间邻居,总表,19900001,,???,？？？,???,???,???,???,???,???,????,???,?,???,؟؟؟,???,???
4,支线房间邻居,总表,19900002,,Alice,爱丽丝,アリス,Alice,Alice,艾莉絲,Alice,Алиса,Alice,Alice,앨리스,Alice,أليس!,Alice,อลิซ


In [17]:
# 活动剧情对话
time.sleep(60)
juqingduihua_spread = read_spreadsheet(juqingduihua)
# juqingduihua_spread['key'] = ''
juqingduihua_spread = set_columns(juqingduihua_spread)
juqingduihua_spread = drop_trash(juqingduihua_spread)
print(len(juqingduihua_spread))
juqingduihua_spread.head(2)

numbers of Sheets: 7
Length: 346
326


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,活动剧情对话,#译文总表,FALSE,A_BP3EventStart_1,"Hi, Alice! Guess what? I got an invitation fro...",,ねえアリス聞いて！フェリペからマッチビュー映画祭に招待されたの！,Hey Alice! Du wirst es nicht erraten! Ich habe...,"Salut, Alice ! Tu sais quoi ? J’ai reçu une in...",嗨，艾莉絲姐姐！你知道嗎，費利佩邀請了我參加麻雀鎮第一屆電影節呢！,"Ciao, Alice. Indovina? Felipe mi ha invitato a...","Привет, Алиса! А знаешь, что? Филипп пригласил...","¡Hola, Alice! Adivina... ¡Me llegó una invitac...","Oi, Alice! Adivinha só? Eu recebi um convite d...","안녕하세요, 앨리스! 이번에 처음 열리는 매치뷰 영화 축제 초대장을 펠리페한테 받았어요!",Selam Alice! Tahmin et ne oldu? Eşseyir’de ilk...,مرحبا يا «أليس»! خمني ماذا؟ تلقيت دعوة من فيلي...,"Hai, Alice! Tahu tidak? Aku mendapatkan undang...",สวัสดีค่ะพี่อลิซ! รู้ไหมคะ หนูได้รับคำเชิญจากเ...
6,活动剧情对话,#译文总表,FALSE,A_BP3EventStart_2,"I’m gonna make a movie about my favorite book,...",,大好きな『灰の家』を映画にするんだ！,Ich werde einen Film über mein Lieblingsbuch m...,Je vais produire un film en me basant sur mon ...,我肯定要把我最喜歡的書《塵土之屋》拍成一部電影！,Realizzerò un film sul mio libro preferito: Il...,Я сниму фильм по моей любимой книге — «Пепельн...,Voy a hacer una película acerca de mi libro fa...,"Vou fazer um filme sobre meu livro favorito, C...",그래서 제가 좋아하는 책 “재의 집”을 바탕으로 영화를 만들려고요!,En sevdiğim kitap House of Ashes hakkında bir ...,سأقوم بعمل فيلم عن كتابي المفضل هاوس أوف آشيز ...,"Aku akan membuat film tentang buku favoritku, ...",หนูจะสร้างภาพยนตร์เกี่ยวกับหนังสือเล่มโปรดของห...


In [18]:
# 活动AI气泡
time.sleep(60)
huodongqipao_spread = read_spreadsheet(huodongqipao)
# huodongqipao_spread['key'] = ''
huodongqipao_spread = set_columns(huodongqipao_spread)
huodongqipao_spread = drop_trash(huodongqipao_spread)
print(len(huodongqipao_spread))
huodongqipao_spread.head(2)

numbers of Sheets: 6
Length: 413
384


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,活动AI气泡,#译文总表,80135,FALSE,"Oh, my word! You can see the little dragon in ...",,わぁ、すごいわ。小さなドラゴンが見える！,Ach du meine Güte! Man kann den kleinen Drache...,Ça alors ! On peut voir un petit dragon à l’in...,哇塞，這裡面還能看到小小龍呢！,"Oh, cielo! Si vede un piccolo drago all’interno!",Батюшки мои! Там даже есть малыш-дракон!,"¡Oh, te doy mi palabra! ¡Puedes ver el dragón ...","Oh, minha nossa! Dá pra ver o pequeno dragão a...","와, 깜짝이야! 저 안에서 작은 용이 보여!",Vay canına! Oradaki küçük ejderhayı görebiliyo...,يا إلهي! يمكنك رؤية التنين الصغير هناك!,Ya ampun! Kamu bisa lihat naga kecil di sana!,โอ้ ไม่น่าเชื่อ! เรามองเห็นมังกรตัวน้อยในนั้นไ...
6,活动AI气泡,#译文总表,80136,FALSE,"This set looks amazing, Gena! Did Taylor make it?",,このセットすごいわね、ジーナ。テイラーが作ったの？,"Dieses Set sieht fantastisch aus, Gena! Hat Ta...","Cet ensemble est incroyable, Emma ! Est-ce que...",這拍攝場景也太厲害了吧，潔娜！是不是你媽咪做的？,"Questo set è stupendo, Gena! C’è la mano di Ta...","Какая изумительная декорация, Джина! Это Тейло...","¡Este set se ve increíble, Gena! ¿Lo hizo Taylor?","Esse set está incrível, Gena! Foi a Taylor que...","세트장이 무지 근사하다, 제나야! 테일러가 만들어 줬니?",Bu set harika görünüyor Gena! Taylor başardı mı?,هذه المجموعة تبدو مذهلة يا جينا! هل نجح تايلور؟,"Set ini terlihat luar biasa, Gena! Apakah Tayl...",นี่ดูน่าทึ่งมากเลยจีน่า! เทย์เลอร์เป็นคนทำใช่ไ...


In [19]:
# UI文字
time.sleep(60)
ui_spread = read_spreadsheet(ui)
ui_spread['key'] = ''
ui_spread = set_columns(ui_spread)
ui_spread = drop_trash(ui_spread)
print(len(ui_spread))
ui_spread.head(2)

总表
Length: 2019
2013


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
1,UI文字,总表,1,,OK,确 认,確認,OK,Confirmer,確認,OK,ОК,OK,OK,확인,Onayla,موافق,OK,ตกลง
2,UI文字,总表,2,,Cancel,取 消,キャンセル,Abbrechen,Annuler,取消,Annulla,Отмена,Cancelar,Cancelar,취소,İptal et,إلغاء,Batal,ยกเลิก


In [20]:
# 邮件文本
youjian_spread = read_spreadsheet(youjian)
youjian_spread['key'] = ''
youjian_spread = set_columns(youjian_spread)
youjian_spread = drop_trash(youjian_spread)
print(len(youjian_spread))
youjian_spread.head(2)

总表
Length: 26
24


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
2,邮件文本,总表,10,,"Dear new owner,",亲爱的庄园主，,新しい家主さんへ、,"Liebe*r Neubesitzer*in,","Cher nouveau propriétaire,",親愛的新屋主，,"Caro nuovo proprietario,","Дорогой новый владелец,",A la atención de quien haya heredado la casa:,"Caro novo proprietário,","친애하는 새 집주인님께,","Sayın yeni sahibimiz,",المالك الجديد المحترم،,"Kepada pemilik baru,",เรียน เจ้าของคนใหม่
3,邮件文本,总表,11,,Welcome to the neighborhood! On behalf of the ...,欢迎您的到来！这是您的邻居们为您准备的礼物，请笑纳。,ご近所へようこそ！コミュニティを代表し、新しい家をお祝いするギフトを贈ります。,Willkommen in unserem Viertel! Im Namen der Na...,Nous sommes ravis de vous accueillir dans le q...,歡迎來到這個社區！在此，謹代表整個社群，獻上一份歡迎禮。,La nostra comunità ti porge un caloroso benven...,Добро пожаловать в наш район! От лица сообщест...,¡Te damos la bienvenida al barrio! De parte de...,Bem-vindo à vizinhança! Estamos mandando um pr...,마을에 오신 걸 환영합니다! 주민들을 대표해 저희가 집들이 선물을 준비했어요.,"Mahalleye hoş geldiniz! Topluluk adına, işte y...",مرحبا بك في الحي! أتقدم إليكم بالنيابة عن المج...,"Selamat datang! Atas nama komunitas, ini hadia...",ยินดีต้อนรับสู่ย่านที่อยู่อาศัยนี้! ในฐานะตัวแ...


In [21]:
# 气泡
# time.sleep(60)
qipao_spread = read_spreadsheet(qipao)
qipao_spread['key'] = ''
qipao_spread['id'] = ''
qipao_spread = qipao_spread[['file_name', 'sheet_name', 'id', 'key', 'en', 'cn', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'pt', 'kr', 'tr', 'ar', 'id', 'th']]
qipao_spread.columns = ['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'id', 'th']
qipao_spread = drop_trash(qipao_spread)
print(len(qipao_spread))
qipao_spread.head(2)

总表
Length: 1548
1548


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,id,th
0,气泡,总表,,,英语,简体中文,日语,德语,法语,繁体中文,意大利语,俄语,西班牙语,葡萄牙语,韩语,土耳其语,阿拉伯语,,泰语
1,气泡,总表,,,This place is like a dream!,像梦一样美好的地方！,まるで夢のような場所ね！,Es ist traumhaft hier!,On se croirait dans un rêve !,這地方真的好夢幻啊！,Questo posto è un sogno!,Я словно в сказку попала!,¡Este lugar es todo un sueño!,Esse lugar parece um sonho!,이곳은 마치 꿈속 같아!,Burası rüya gibi!,هذا المكان مثل الحلم!,,ที่นี่เหมือนกับความฝันเลย!


In [22]:
# 三选家具名
sanxuanjiaju_spread = read_spreadsheet(sanxuanjiaju)
sanxuanjiaju_spread = sanxuanjiaju_spread[['file_name', 'sheet_name', 'id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', '#ar', '#id', '#th']]
sanxuanjiaju_spread.columns = ['file_name', 'sheet_name', 'id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th']
sanxuanjiaju_spread = set_columns(sanxuanjiaju_spread)
sanxuanjiaju_spread = drop_trash(sanxuanjiaju_spread)
print(len(sanxuanjiaju_spread))
sanxuanjiaju_spread.head(2)

numbers of Sheets: 17
Length: 1181
1135


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,三选家具名,Language_FurnitureName_W1A1#广场区家具名,10101101100200,,Coronado,明亮黄,コロナド,Coronado,Coronado,科羅納多,Coronado,Коронадо,Coronado,Coronado,코로나도,Coronado,كورونادو,Coronado,โคโรนาโด้
6,三选家具名,Language_FurnitureName_W1A1#广场区家具名,10101101100300,,Rocaille,典雅白,ロカイエ,Rocaille,Rocaille,石貝飾,Rocaille,Рокайль,Rocaille,Rocaille,로카이유,Rocaille,روكايل,Rocaille,ร็อคไคล์


# Merge Files

In [23]:
spread_sheet_list = [zhuxian_spread, renwuqipao_spread, sanxuanqipao_spread, renwuming_spread, renwuzhangjie_spread, renwumingcheng_spread, jiajuming_spread, renwuduihua_spread, 
                    zhixianrenwuming_spread, zhixianlinju_spread, juqingduihua_spread,huodongqipao_spread, ui_spread, youjian_spread, qipao_spread, sanxuanjiaju_spread]

In [24]:
whole = pd.DataFrame()
for sheet in spread_sheet_list:
    whole = whole.append(sheet)
print(len(whole))

29462


In [25]:
whole.head()

,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th,id
2,主线任务对话,#译文总表,NaN,程序用,英语,简体中文,日语,德语,法语,繁体中文,意大利语,俄罗斯语,西班牙语,葡萄牙语,韩语,土耳其语,阿拉伯语,印尼语,泰语,NaN
3,主线任务对话,#译文总表,NaN,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,NaN
5,主线任务对话,#译文总表,NaN,T_OrchardBuildpathfinal_11,Just <color=#3a9108>assemble the three pieces<...,,<color=#3a9108>3つのパーツを組み立てて</color>、車輪を回すだけじゃ！...,<color=#3a9108>Setzt die drei Stücke einfach z...,Il vous suffit d’<color=#3a9108>assembler les ...,把<color=#3a9108>三塊拼起來</color>，再把輪子轉一下！這就是開啟鐵門的...,<color=#3a9108>Metti insieme i tre pezzi</colo...,Давай <color=#3a9108>соберем все три элемента<...,¡Ensambla las tres piezas y <color=#3a9108>gir...,É só <color=#3a9108>montar as três peças</colo...,<color=#3a9108>세 조각을 합쳐서</color> 바퀴를 돌려라! 그게 대...,<color=#3a9108>Üç parçayı bir araya getir</col...,كل ما عليك هو <color=#3a9108>تجميع القطع الثلا...,<color=#3a9108>Susun saja ketiga bagiannya</co...,แค่<color=#3a9108>ประกอบทั้งสามชิ้นส่วน</color...,NaN
6,主线任务对话,#译文总表,NaN,E_DockDecoratearchespre_2,A little tracker would do the trick.,,小さなトラッカーがあれば大丈夫だ。,Ein kleiner Tracker wird den Unterschied machen.,Un petit traqueur fera l’affaire.,加個追蹤器，肯定沒問題。,Un piccolo localizzatore sarà sufficiente.,Незаметный маячок будет очень кстати.,Un pequeño rastreador hará el trabajo.,Um pequeno rastreador deve dar conta do recado.,추적기를 하나 달면 되겠어.,Küçük bir takip cihazı işe yarar.,تتبع أثر العملية قليلاً سيفي بالغرض.,Pelacak kecil cukup untuk melakukannya.,ใช้เครื่องติดตามสักหน่อยน่าจะได้ผล,NaN
7,主线任务对话,#译文总表,NaN,E_DockDecoratearchespre_3,"So even if that island's in Timbuktu, I'll be ...",,たとえ島がアフリカにあろうと、どこに向かっているのかがわかるはずだ！,"Selbst wenn diese Insel in Timbuktu liegt, so ...","Comme ça, même si cette île est à Timbuktu, je...",就算他們把船開到鳥不生蛋的地方，我都能知道！,"Così anche se quell'isola si trova a Timbuktu,...","Пусть этот остров находится хоть в Тимбукту, н...","¡Así, incluso si la isla es en Tombuctú, sabré...","Então mesmo que aquela ilha seja em Timbuktu, ...",그러면 그 섬이 아무리 외딴 곳에 있더라도 놈들이 향한 곳을 알 수 있겠지!,Yani o ada Timbuktu’da olsa bile nereye gittik...,لذا حتى لو كانت تلك الجزيرة في تمبكتو، فسأكون ...,"Jadi walaupun pulau itu ada di Timbuktu, aku b...",ถึงแม้เกาะนั่นจะอยู่ในแดนลึกลับแค่ไหน ฉันก็จะร...,NaN


In [26]:
whole.columns

Index(['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de',
       'fr', 'tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th',
       'id'],
      dtype='object')

In [27]:
whole = whole[['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de','fr', 'tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th']]
whole.columns = ['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'id', 'th']

In [28]:
con = sqlite3.connect("/mnt/e/Desktop/1/b本地化/Nova文字表/nova_texts.sqlite")
c = con.cursor()
c.execute("drop table nova_texts")
c.execute("create table nova_texts(file_name text, sheet_name text, text_id text, key text, en text, sc text, jp text, de text, fr text, tc text, it text, ru text, es text, po text, kr text, tu text, ar text, id text, th text)")
con.commit()
whole.to_sql("nova_texts", con, if_exists="append",index=False)
con.close()

# Check

In [56]:
con = sqlite3.connect("/mnt/e/Desktop/1/b本地化/Nova文字表/nova_texts.sqlite")
SQL="""
select * from nova_texts
"""
df = pd.read_sql_query(SQL,con)
con.close()
df.head()

,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,id,th
0,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_1,When you told me you inherited this mansion fr...,呃，虽说你叔叔查尔斯把这个庄园留给你是件天大的好事，可这房子……,あなたがチャールズ叔父さんからこの屋敷を受け継いたって聞いたとき、信じられなかったわ。でも…,"Als du mir erzählt hast, dass du diese Villa v...",Quand tu m’as dit que ton oncle Charles t’avai...,你上次跟我說你從查爾斯叔叔繼承了一棟豪宅，我本以爲這是個天大的好事，但...,Quando mi hai detto che avevi ereditato questa...,"Я даже не поверила сначала, когда узнала, что ...",Cuando me contaste que habías heredado la mans...,Quando você me disse que herdou essa mansão do...,"네가 이 저택을 삼촌 찰스 아저씨에게서 물려받았다고 했을 때는, 너무 꿈같은 얘기라...",Bu köşkün sana Charles Amcandan kaldığını söyl...,عندما أخبرتني أنك قد ورثت هذا القصر من عمك تشا...,Waktu kamu cerita bahwa kamu mewarisi rumah in...,ตอนที่เธอบอกฉันว่าได้รับคฤหาสน์นี้เป็นมรดกจากล...
1,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_2,"...But since we're already here, we should rev...",不过既来之，则安之，我们一定能把这个庄园修缮好的！,...せっかくここまで来たんだもの！この屋敷を私達の手で蘇らせましょう！,"... Aber da wir jetzt schon mal hier sind, sol...",... nous y voilà ! Autant rénover cette belle ...,...反正我們人都來了，是時候給這棟豪宅好好翻新一下！,"... Ma già che ci siamo, dovremmo ridarle vita!","...Но раз уж мы тут, надо его восстановить!","...Ya que estamos aquí, ¡deberíamos revivir la...","...Mas como já estamos aqui, deviamos reviver ...","...이미 여기까지 왔으니까, 우리가 저택을 되살려 보자!","...Madem zaten buradayız, köşkü yeniden canlan...",...لكن بما أننا بالفعل هنا، علينا أن نجدد القصر!,"... Tapi karena kita sudah ada di sini, kita h...",...แต่ไหนๆ ก็มาถึงที่่นี่แล้ว เรามาฟื้นฟูคฤหาส...
2,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_3,Let's start by <color=#3a9108>removing all thi...,就先从<color=#3a9108>清理垃圾</color>开始吧！,まずは、<color=#3a9108>雑草を片付け</color>ましょうか。,"Beginnen wir, indem wir <color=#3a9108>all die...",Commençons par <color=#3a9108>retirer tout ce ...,我們先<color=#3a9108>清理一下這些垃圾</color>。,Iniziamo <color=#3a9108>rimuovendo tutta quest...,Давай сначала <color=#3a9108>уберем весь этот ...,Comencemos por <color=#3a9108>recoger toda est...,Vamos começar <color=#3a9108>removendo todo es...,우선 <color=#3a9108>이 쓰레기부터 전부 치우자</color>.,<color=#3a9108>Tüm bu çöpleri kaldırarak</colo...,هيا نبدأ <color=#3a9108>بالتخلص من كل هذه النف...,Ayo kita mulai dengan <color=#3a9108> membuang...,เริ่มต้นด้วยการ<color=#3a9108>กำจัดขยะทั้งหมด<...
3,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareRemoveJunk_1,This isn't public property! Get out!,嘿！这儿是私人住所！谁让你们进来的？,おい！ここは所有地じゃぞ！出て行け！,Das hier ist kein öffentliches Gelände! Versch...,C’est privé ici ! Allez-vous-en !,啊！出去出去！這裡可不是什麽公共場所！,"Questa è proprietà privata, andate via!","Эй, это частная собственность! Проваливайте!",¡Esto es propiedad privada! ¡Aléjense!,Isso não é propriedade pública! Saia daqui!,여기 사유지입니다! 나가요!,Burası özel bir mülk! Defolun buradan!,هذه ليست ملكيةً عامةً! اخرجوا!,Ini bukan milik umum! Keluar!,ที่นี่ไม่ใช่ที่สาธารณะ! ออกไปซะ!
4,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareRemoveJunk_2,"Oh my goodness, Uncle Alan! It's me, Alice and--",哇，艾伦叔叔！是我啊，爱丽丝！还有这位是--,まあ、アラン叔父さん！私よ、アリスと…,"Ach du meine Güte! Onkel Alan! Ich bin‘s, Alic...","Oh mon dieu, oncle Alain ! C’est moi, Alice ! ...",天啊，阿朗叔叔！是我、艾莉絲還有～,"Oh, cielo, zio Alan! Sono io, Alice, e...","Надо же, дядя Алан! Это я, Алиса, и...","Oh Dios mío, ¡tío Alan! Soy yo, Alice y...","Meu deus, Tio Alan! Sou eu, a Alice e...","세상에, 앨런 아저씨! 저 앨리스예요. 이쪽은--","Tanrım, Alan Amca! Alice ben, hatırlamadın mı?...",يا إلهي، العم ألن! إنه أنا «أليس»، و--,"Ya Tuhan, Paman Alan! Ini aku, Alice dan--",โอ้แย่แล้ว ลุงอลันคะ! นี่หนูเอง อลิซกับ--


In [57]:
# df.to_excel()

In [33]:
con = sqlite3.connect("/mnt/e/Desktop/1/b本地化/Nova文字表/nova_texts.sqlite")
SQL="""
select 
file_name, 
sheet_name, 
text_id, 
key,
en, 
sc
from nova_texts
where en like '%ffairs%'
"""
df = pd.read_sql_query(SQL,con)
con.close()
df.head()

,file_name,sheet_name,text_id,key,en,sc
0,UI文字,总表,87,,Do you like Garden Affairs? Please rate us and...,你喜欢Garden Affairs吗？请为我们打分并提出宝贵的意见吧。
1,UI文字,总表,103,,Check your emails regularly to stay updated on...,定期查看邮件，了解小镇最新讯息。
2,UI文字,总表,393,,Follow Garden Affairs on social media to get t...,关注 Garden Affairs 社交账号，获取最新游戏讯息。
3,UI文字,总表,399,,Do you like Garden Affairs? Rate us in the app...,喜欢 Garden Affairs 吗？来应用商店为我们评分吧。
4,UI文字,总表,622,,Would you recommend Garden Affairs to your fri...,你愿意给Garden Affairs打多少分？


In [3]:
df.columns

Index(['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de',
       'fr', 'tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'id', 'th'],
      dtype='object')

In [9]:
from langdetect import detect
detect("Ein, zwei, drei, vier")

'de'

In [35]:
langlist = ['jp', 'de','fr','it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'id', 'th']
textchecklist = []
for lang in langlist:
    sublist = []
    for text in df[lang]:
        try:
            sublist.append(detect(text))
        except:
            sublist.append('Not_Text')
    textchecklist.append(sublist)

In [38]:
len(textchecklist), len(langlist)

(12, 12)

In [37]:
len(textchecklist[0])

14967

In [39]:
num = 0
for lang in langlist:
    col_name = lang + '_check'
    df[col_name] = textchecklist[num]
    num += 1

In [41]:
df.columns

Index(['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de',
       'fr', 'tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'id', 'th',
       'jp_check', 'de_check', 'fr_check', 'it_check', 'ru_check', 'es_check',
       'po_check', 'kr_check', 'tu_check', 'ar_check', 'id_check', 'th_check'],
      dtype='object')

In [42]:
df = df[['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc','tc', 
'jp', 'jp_check', 'de','de_check',
'fr','fr_check', 'it','it_check',  
'ru', 'ru_check', 'es', 'es_check', 'po','po_check', 'kr','kr_check', 'tu','tu_check',  'ar','ar_check',  'id', 'id_check','th','th_check']]

In [43]:
df.head(2)

,file_name,sheet_name,text_id,key,en,sc,tc,jp,jp_check,de,...,kr,kr_check,tu,tu_check,ar,ar_check,id,id_check,th,th_check
0,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_1,When you told me you inherited this mansion fr...,呃，虽说你叔叔查尔斯把这个庄园留给你是件天大的好事，可这房子……,你上次跟我說你從查爾斯叔叔繼承了一棟豪宅，我本以爲這是個天大的好事，但...,あなたがチャールズ叔父さんからこの屋敷を受け継いたって聞いたとき、信じられなかったわ。でも…,ja,"Als du mir erzählt hast, dass du diese Villa v...",...,"네가 이 저택을 삼촌 찰스 아저씨에게서 물려받았다고 했을 때는, 너무 꿈같은 얘기라...",ko,Bu köşkün sana Charles Amcandan kaldığını söyl...,tr,عندما أخبرتني أنك قد ورثت هذا القصر من عمك تشا...,ar,Waktu kamu cerita bahwa kamu mewarisi rumah in...,id,ตอนที่เธอบอกฉันว่าได้รับคฤหาสน์นี้เป็นมรดกจากล...,th
1,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_2,"...But since we're already here, we should rev...",不过既来之，则安之，我们一定能把这个庄园修缮好的！,...反正我們人都來了，是時候給這棟豪宅好好翻新一下！,...せっかくここまで来たんだもの！この屋敷を私達の手で蘇らせましょう！,ja,"... Aber da wir jetzt schon mal hier sind, sol...",...,"...이미 여기까지 왔으니까, 우리가 저택을 되살려 보자!",ko,"...Madem zaten buradayız, köşkü yeniden canlan...",tr,...لكن بما أننا بالفعل هنا، علينا أن نجدد القصر!,ar,"... Tapi karena kita sudah ada di sini, kita h...",id,...แต่ไหนๆ ก็มาถึงที่่นี่แล้ว เรามาฟื้นฟูคฤหาส...,th


In [55]:
df_drop_sametxts = df.drop(df[(df['en']==df['sc']) & (df['en']==df['jp'])& (df['en']==df['de'])& (df['en']==df['fr'])& (df['en']==df['tc'])& (df['en']==df['it'])& (df['en']==df['ru'])& (df['en']==df['es'])& (df['en']==df['po'])& (df['en']==df['kr'])& (df['en']==df['tu'])& (df['en']==df['ar'])& (df['en']==df['id'])& (df['en']==df['th'])].index)

In [58]:
len(df), len(df_drop_sametxts)

(14967, 14588)

In [59]:
df_drop_sametxts.to_excel('/mnt/e/Downloads/check_list.xlsx',index=False)

In [62]:
df_drop_sametxts[(df_drop_sametxts['po_check']=='es')&(df_drop_sametxts['es_check']=='pt')].to_excel('/mnt/e/Downloads/check_list2.xlsx',index=False)

In [ ]:
df['en']==df['sc'] & df['en'] == df['jp'] & df[, 'de',
       'fr', 'tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'id', 'th',

# Historical Translation Records

In [ ]:
# whole： https://drive.google.com/drive/folders/1Uq9c9OP6rvOkv_N8XUrYxVNYanBb6Y_N

# 已翻译过的内容：https://docs.google.com/spreadsheets/d/1buAQ41UtnyBZze7ILo19QWqfbslA_8G8jTa10zrM11c/edit#gid=0


# AGM Translation:
# AGM多语言表_SDK20200622 = https://docs.google.com/spreadsheets/d/1wIQ8ow6HVyo1mg2FJdc2fyX56bMUVwfN/edit#gid=1763315832
# AGM多语言表_20210115 = https://docs.google.com/spreadsheets/d/17yz7AJgG-CF0NagPdeDElqDE3_p7U0zG/edit#gid=633467999
# AGM多语言表_20201222 = https://docs.google.com/spreadsheets/d/19zrW1N8-gs_cxckPJMIYU7eadkwz0mc9/edit#gid=1310282413
# AGM多语言表_CG20200506_final = https://docs.google.com/spreadsheets/d/1-Bewbru2qTK5vnjkCS5g4TyRKXJ2Kggk/edit#gid=244612586
# AGM多语言表_20201022 = https://docs.google.com/spreadsheets/d/1Eek1vaEufmCBR5UEkN102-7-E4mVPhfO/edit#gid=1444700303
# AGMtranslated_多语言表_20201012 = https://docs.google.com/spreadsheets/d/1eCBwsfNnTEvBHONSm6eKNsz8w9fcTpfr/edit#gid=1570199498
# AGM多语言表_UI20200318_完了 = https://docs.google.com/spreadsheets/d/17ItMKigxKMylMhftnEAU-lkWxWv61e8L/edit#gid=1241765642
# AGM納品200918_多语言表_20200831 = https://docs.google.com/spreadsheets/d/1CK1BOcEBQ6j9vs2pRGH78M-JuAl-LerD/edit#gid=522550629
# ————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
# ————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
# ————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
# GFeature = https://docs.google.com/spreadsheets/d/1FCIEsdW-QYOfjp5rWbSercxFXDj0dVL5/edit#gid=1437206883
# LevelvsStage = https://docs.google.com/spreadsheets/d/1ch_aPcET2T1gBu2JEhb1blp_OgqTSzfu4i-r1f4iIdA/edit#gid=574929400
# UI_REVIEW = https://docs.google.com/spreadsheets/d/1274dxSXNh26aBKIWuXa9Pqa8jtNnl1Zd/edit#gid=850630259
# UI_Review_Arial = https://docs.google.com/spreadsheets/d/127_I6xywPH0lP9TcMKMV7j5sZWrKOXAC/edit#gid=1902315134
# UI_CAPITALS = https://docs.google.com/spreadsheets/d/15O2ondGgzUzCgLq52OWsBMeqlB7UskNKRWCxH1jH7qQ/edit#gid=1301004822
# StoreIntro_20220615 = https://docs.google.com/spreadsheets/d/1_KG2yF7jN4RWR2Mgima0oJEu8X50gxbvWDkPhpnGr8A/edit#gid=1528809237
# NEW4_Catch_UP_UI = https://docs.google.com/spreadsheets/d/1u7NuH8K1ZFwsa3_G2kGmvoUK3g38-1QujZxRtdXPEFc/edit#gid=700452977
# NEW4_Catch_UP_REST = https://docs.google.com/spreadsheets/d/1_KhR-hJLMhuTaPtp5IFjwWE57Vn0CH4AkV7Vsictspo/edit#gid=1171393775
# NEW4_Catch_Up_MAIN = https://docs.google.com/spreadsheets/d/1Ed8w-cqYvpxV3uNoVfcXRKLRUbwF8DglxcrSgQCm6kQ/edit#gid=1753905361
# Perfect_Life = https://docs.google.com/spreadsheets/d/1uwX4a4vG7N-xBSkqp8CztwU3OZNmIV0HgtG6DYIkOsM/edit#gid=1570615355
# A1_3 = https://docs.google.com/spreadsheets/d/1GE0sO5yays_Etg0okekR0xBks81E1QSC_fwseaGGlmY/edit#gid=0
# A1_3_pt1 = https://docs.google.com/spreadsheets/d/1xgpRJWRXIjX5tJjT3cs80-8-aOKPiXJo/edit#gid=403249415
# A1_3_complete_almost = https://docs.google.com/spreadsheets/d/1DmNSaTA4xyX4cg4RGUIHxKrnZ6BN8qwA/edit#gid=373985788
# A4_en_C = https://docs.google.com/spreadsheets/d/1WBA9OBJKplPa1lhK3W4AAjOWMnQ1TX8v/edit#gid=1443474189
# A4_REVIEW_ARIAL = https://docs.google.com/spreadsheets/d/1lwss7r7zdbklVjX3VObG_8PouwI3zRjB/edit#gid=158309738
# A4_JAKOZHHant = https://docs.google.com/spreadsheets/d/1-gCIV9Rnjyzd2PbvrFVH6pToYQyMYqPl/edit#gid=1658563656
# A5 = https://docs.google.com/spreadsheets/d/1w25wa5hvhILmIXYxyybyGd3FVyFSgcbWEjiO63FL3ZU/edit#gid=0
# A5_REVIEW10pct = https://docs.google.com/spreadsheets/d/14dPTC8dBm8z5wg6grp6YXMBBRlA1KWPC/edit#gid=1875772470
# A5_UI = https://docs.google.com/spreadsheets/d/1Hnug8dYrmI4JwKHiWFOs7X0RPVuzieC7/edit#gid=1974769509
# A5_剧情 = https://docs.google.com/spreadsheets/d/17jcrcVQYr7oEzBwDctS8i7BAjVNeqKS0/edit#gid=1050707207
# A6_REVIEW = https://docs.google.com/spreadsheets/d/1IyjURN6K9s5nRU7ABQBW2rcI841MO0r7/edit#gid=144223150
# A6_剧情 = https://docs.google.com/spreadsheets/d/1ksvSZNibA_MY9NzqBsqrDqgT0Vhr6l49/edit#gid=543068104
# A7_REVIEW = https://docs.google.com/spreadsheets/d/19isGwC-XQNgBZwqBw9tBVlJiyfUW-DG6/edit#gid=976872718
# A7_UPLOAD = https://docs.google.com/spreadsheets/d/1H21G3KCUOFCM1f0OuWJncUpdaL6Nvhnj/edit#gid=285802455
# A8_REVIEW = https://docs.google.com/spreadsheets/d/1OTUiNhy21wmHQW4uAP7AYqS1Yjb8jckv/edit#gid=1703127275
# A8_Upload = https://docs.google.com/spreadsheets/d/1EcFA5bD2yYaOrGvpYs4zURnEnIKla-f1/edit#gid=949588301
# NEW4_A1_A3 = https://docs.google.com/spreadsheets/d/19LzmYQx7GSQ9daJ65j2prrFcVZJHiKce/edit#gid=271273286
# NEW4_A4 = https://docs.google.com/spreadsheets/d/1B91iKhNjhsXz_JlW1ZhkHYYUZhHGloEM/edit#gid=310086593
# NEW4_A5 = https://docs.google.com/spreadsheets/d/1BH8fluGluHwCmUMP51uRu_7ZUdxyCACE/edit#gid=1811312064
# NEW4_A6 = https://docs.google.com/spreadsheets/d/1BPQasj0uLLlKPMK0ILklzXBFywoXYZPj/edit#gid=458136847
# NEW4_A7 = https://docs.google.com/spreadsheets/d/1BPXYX_DtDnPn9hagsd_M30ny6DzoxHhI/edit#gid=1182120524
# NEW4_A8 = https://docs.google.com/spreadsheets/d/1BPjp00ppXl3FPF1Pq56KAcwdJs9MVbjr/edit#gid=865630099
# NEW4_A9 = https://docs.google.com/spreadsheets/d/1BUzR_yK-4uM1QnpJ6EwrAlNHBj-VHk6F/edit#gid=1347358105
# NEW4_A10 = https://docs.google.com/spreadsheets/d/1BcWGWPX2h4fDIF4xk7PWbbRcPhi-y853/edit#gid=983541053
# NEW4_JoeysShop = https://docs.google.com/spreadsheets/d/1BZbphS2-XzQBF58hA8U87nnsqzLE9WWi/edit#gid=883865285
# NEW4_V3EmailBubbles = https://docs.google.com/spreadsheets/d/1BkyHqIXKJXnGizPCaaH_7UAllbBYSJZD/edit#gid=27135767
# NEW4_V4_Text = https://docs.google.com/spreadsheets/d/1Bo-kIo9EFXsFWskxpTxN_08p7r9mV2QH/edit#gid=1584398329
# NEW4_V5_Task1 = https://docs.google.com/spreadsheets/d/1BwPo2skgj8UbyJ2o1hijCNJZbGW1VGr1/edit#gid=322953819
# NEW4_V5_Task2 = https://docs.google.com/spreadsheets/d/1C9t4Fp7rmza6hZvLTzMeGKAqZibD-tlJ/edit#gid=1901312158
# NEW4_V5_CHECKER = https://docs.google.com/spreadsheets/d/11lgSJ3xyjiriKs-9_svLzkDtVF9C53n4/edit#gid=1544620863
# NEW4_V5_T2_CHECKER = https://docs.google.com/spreadsheets/d/19UuMgkvT6T6ezWRjPFpU2Clq1ez8TRS7/edit#gid=1544620863
# NEW4_V5_Task3 = https://docs.google.com/spreadsheets/d/1CDPhmydHAFG1tmfmQAoy2vjFfskJQLSW/edit#gid=1068335772
# NEW4_V6_Task1 = https://docs.google.com/spreadsheets/d/1CF4IA7GawRc-IZ2jWk1HtbokUKujSSOH/edit#gid=1355170182
# NEW4_V6_Task2 = https://docs.google.com/spreadsheets/d/1CIFbw3C_QS3TLn-pOXvnbuDSaBjQZqTB/edit#gid=1777359022
# V7_Task1 = https://docs.google.com/spreadsheets/d/1CKhpx6wFGHNx2JJYB0CQIt6ojliyyYwO/edit#gid=269130524
# V7_Task2 = https://docs.google.com/spreadsheets/d/1COhcNKVpZpwc66DU2yl2mFJWhbbz2cro/edit#gid=1550303953
# V7_Task3 = https://docs.google.com/spreadsheets/d/1CPpTZzCEjipvbcmWPreSP4Aapb_f6UVe/edit#gid=2044690431
# V8_Task1 = https://docs.google.com/spreadsheets/d/1CRe76NaUFGxcfhucGTB0f-AV4XQg60hG/edit#gid=32966401
# V8_Task2 = https://docs.google.com/spreadsheets/d/1CTWoqaaB5nJNTlNG0Dhv4jvVR78tf75I/edit#gid=1982251788
# V8_Task3 = https://docs.google.com/spreadsheets/d/1CVhT0YqaZFTCSpL2nAU3T1wR2EaY2oVJ/edit#gid=1077564597
# V9_Task1 = https://docs.google.com/spreadsheets/d/1mZyaQZwz8sgueMt_yCC9wZFfkZnaXNvmFpxcbkALmLs/edit#gid=0
# V9_Task1_en_C = https://docs.google.com/spreadsheets/d/1CZuxG9HrtmN9VH8AHbrNmi4K9SO7lBuh/edit#gid=213552818
# V9-Task2 = https://docs.google.com/spreadsheets/d/1C_9RvXckoxFxGveJ8EfX45ye17z4DSY3/edit#gid=72281366
# V10_Task1_20211125 = https://docs.google.com/spreadsheets/d/1CbQNtX7IvhZ4RMHLGxnx2u2VHuNjKY9w/edit#gid=1650731684
# V11_Task1 = https://docs.google.com/spreadsheets/d/1CbZlbULaZywRT0RGbYcYEuINy2LRn25a/edit#gid=1896867600
# V11_Task2 = https://docs.google.com/spreadsheets/d/16igUAFMjI_IPXk9N2z-Ex1NhPjaWvPZ9/edit#gid=1229687523
# V12_Task1 = https://docs.google.com/spreadsheets/d/16pgORHujCcG9RxROtitlT9Jb8lPhltKi/edit#gid=1340076466
# V12_Task2 = https://docs.google.com/spreadsheets/d/16x8eUB987SfZlTtLalPvLcs2MfuiQQRI/edit#gid=1223366815
# V12_Task3 = https://docs.google.com/spreadsheets/d/16zd1QwhDGP8eaFzI260E7aie0-xRdyTP/edit#gid=189168987
# V12_Task4 = https://docs.google.com/spreadsheets/d/170hwWK_ML6vzdBPklpWwoUjka_x3nutV/edit#gid=299707911
# V13_Task1 = https://docs.google.com/spreadsheets/d/18vp4E78Oqu7WKUYn0W8Et9FR1C-5uyar/edit#gid=588445702
# V13_Task2 = https://docs.google.com/spreadsheets/d/19RfesaYNExryJUu4c5kjswfenlm17OHM/edit#gid=103604392
# V14_Task1_updated = https://docs.google.com/spreadsheets/d/19z2b4BCTImpwdndRmTAwqbeMS1jK0uTv/edit#gid=1544620863
# V14_Task1_43535updated = https://docs.google.com/spreadsheets/d/1A4gvguRQ8EKEj5voxCh7s5fToyb7Gzi6/edit#gid=1796286032
# V14_Task2 = https://docs.google.com/spreadsheets/d/1A3HsWJ2heI9b9HxkvEXDO-kp3i_n1B5Y/edit#gid=78142038
# V15_T1配置完 = https://docs.google.com/spreadsheets/d/1S6NVnJj6VjbrnWxLzHRbiIDgJZAkIY8A/edit#gid=1544620863
# V15_T1 = https://docs.google.com/spreadsheets/d/1ur9EYpXoleXHtr6BmCWIyvmRcG1ruJ7mo4tUlcCcRy8/edit#gid=1744788529
# V15_T2 = https://docs.google.com/spreadsheets/d/1Bf7bocTkfLOgYejwTGEDt6Y-vb5fbD-m/edit#gid=495151565
# V16_T1 = https://docs.google.com/spreadsheets/d/17HsZflPeMTJe6QFXuGp3yReHNWuZTv3uIUh3TteQsN4/edit#gid=722941420
# V16_T2 = https://docs.google.com/spreadsheets/d/1FZYwl9VC6JmMAmK8TSX0Wqh3yg_tozv7/edit#gid=1418082534
# V17_T1 = https://docs.google.com/spreadsheets/d/1FZ4lYHqJQVMKx7FXj81MDZ54AA9W-Vx-/edit#gid=2108028555
# V17_T2 = https://docs.google.com/spreadsheets/d/1F_i__qwl6Vbc8-xkYKxnb80djFxj-uXs/edit#gid=1047764920
# V17_T3 = https://docs.google.com/spreadsheets/d/1MNOsS74rx2y7LTEkP3XD6HelXfIXAJCkmbqEr646Gic/edit#gid=259246570
# V17_T4 = https://docs.google.com/spreadsheets/d/1FfGqUyYkh5B2t1IpNnX9S7owqsii9BB5/edit#gid=475212308
# V18_T1 = https://docs.google.com/spreadsheets/d/1Fy0q3sg2TUfn63FvLus1tQoDW2fCI4h_/edit#gid=1564900707
# V18_T2 = https://docs.google.com/spreadsheets/d/1cWWTxBn63R_Wb9eI-zoAcZENLaL1M9z9aOnRPn-9Fz4/edit#gid=1122670371
# V18_T3 = https://docs.google.com/spreadsheets/d/1P48MKedl--4DRrEal6yaHHXOZQcX21EjCF-o5jZTWiI/edit#gid=813528355
# V18_T4 = https://docs.google.com/spreadsheets/d/1RPaOn5UZSdR9nVhAx5oHkRrmBclpUhOVuUdMr8u-BWE/edit#gid=840733474
# V19_T1 = https://docs.google.com/spreadsheets/d/11lMhe5O9GJ2aoTzoQ22JWRlRItFwcLUVurmqotkCEpQ/edit#gid=1556378175
# V19_T2 = https://docs.google.com/spreadsheets/d/1XRc7xNc5rOkk2takrFjLMls_Zl4FRQYxXB8mZVv_CMY/edit#gid=1668962918
# V19_T3 = https://docs.google.com/spreadsheets/d/1Qa4xE1XAyfQkbJz8_olQHMV50sqeE7gxeFkuW4mllvQ/edit#gid=122855723
# V20_T1 = https://docs.google.com/spreadsheets/d/16_gj_xHcPeP7wslSncLz3Dwo0lGyYT8WaMUksfaoP44/edit#gid=1964440003
# V20_T2 = https://docs.google.com/spreadsheets/d/1WKFX76YL92Arbo0ku4-jS9HtlBu4f4as8cf54FLwgPE/edit#gid=1123711877
# V20_T3 = https://docs.google.com/spreadsheets/d/1X-XAzKW_i0kWI0Qs9N2se3LaOsHyeYQY2nB9qDyjBuo/edit#gid=1060704600
# V20_T4 = https://docs.google.com/spreadsheets/d/1FzybMhkkrE7hCmI0c28BXMf1vK6A4_vdA30u-_oYdJA/edit#gid=1247258729
# V20_T5 = https://docs.google.com/spreadsheets/d/1Wj3aMhiH7RFTaXVEZon9f6XjThnA6NlveSrQgwUmEBw/edit#gid=1588181460
# V20_T6 = https://docs.google.com/spreadsheets/d/1cLhJAmvimybf-OvsiwhjQQe9jMbkGBn9DXtCxDnHyk0/edit#gid=1510143309
# V21_T1 = https://docs.google.com/spreadsheets/d/1O5sSqiE60La0tU4qZavS7Iy5jhncF403BC2EjfTj0p8/edit#gid=387533947
# V21_T2 = https://docs.google.com/spreadsheets/d/11_tk98ItXporzegexUhTQSpQ6T43Z5TCRx_xk0zMBt0/edit#gid=2053444364
# V22_T1 = https://docs.google.com/spreadsheets/d/1p7DfT085_u0JYpKaCVbZ-DJp2svWoZw_x6GsXrki74Q/edit#gid=191548985

In [3]:
# https://docs.google.com/spreadsheets/d/1buAQ41UtnyBZze7ILo19QWqfbslA_8G8jTa10zrM11c/
key = '1buAQ41UtnyBZze7ILo19QWqfbslA_8G8jTa10zrM11c'

In [4]:
response = google_account.open_by_key(key)
sheet = response.get_worksheet(0)
sheet_df = pd.DataFrame(sheet.get_all_records())
df = sheet_df.copy()

In [43]:
df = sheet_df.copy()

In [44]:
len(sheet_df), len(df)

(31999, 31999)

In [46]:
print("sheet_df Length: {}".format(len(sheet_df)))
print("df Length: {}".format(len(df)))

sheet_df Length: 31999
df Length: 31999


In [47]:
df['word_count'] = df['英语'].str.split().str.len()

In [53]:
df.columns

Index(['FileName', 'Note', '英语', '简中', '日语', '德语', '法语', '繁中', '意语', '俄语',
       '西语', '葡语', '韩语', '土语', '阿语', '印尼语', '泰语', 'Link', 'ID', 'max_length',
       'word_count'],
      dtype='object')

In [54]:
len(df.columns)

21

In [51]:
df = df[['FileName', 'Note', '英语', '简体中文', '日语', '德语', '法语', '繁体中文', '意大利语', '俄罗斯语', '西班牙语', '葡萄牙语', '韩语','土耳其语', '阿拉伯语', '印尼语', '泰语', 'Link', 'ID', 'MAX LENGTH', 'word_count']]
df.columns = ['FileName', 'Note', '英语', '简中', '日语', '德语', '法语', '繁中', '意语', '俄语', '西语', '葡语', '韩语','土语', '阿语', '印尼语', '泰语','Link', 'ID','max_length', 'word_count']

In [41]:
con.close()

In [52]:
con = sqlite3.connect("/mnt/e/Desktop/1/b本地化/Nova文字表/nova_texts.sqlite")
c = con.cursor()
c.execute("drop table nova_texts_translated")
c.execute("create table nova_texts_translated(FileName text, Note text, 英语 text, 简中 text, 日语 text, 德语 text, 法语 text, 繁中 text, 意语 text, 俄语 text, 西语 text, 葡语 text, 韩语 text, 土语 text, 阿语 text, 印尼语 text, 泰语 text , Link text, ID text, max_length text, word_count text)")
con.commit()
df.to_sql("nova_texts_translated", con, if_exists="append",index=False)
con.close()

In [55]:
df.columns

Index(['FileName', 'Note', '英语', '简中', '日语', '德语', '法语', '繁中', '意语', '俄语',
       '西语', '葡语', '韩语', '土语', '阿语', '印尼语', '泰语', 'Link', 'ID', 'max_length',
       'word_count'],
      dtype='object')

In [75]:
con = sqlite3.connect("/mnt/e/Desktop/1/b本地化/Nova文字表/nova_texts.sqlite")
SQL="""
SELECT distinct 英语
FROM nova_texts_translated
"""
df = pd.read_sql_query(SQL,con)
con.close()
df.head()

,英语
0,When you told me you inherited this mansion fr...
1,"But since we're here already, we should revive..."
2,Let's start by <color=#3a9108>removing all thi...
3,Remove junk
4,Much better!


In [76]:
df['英语'].str.split().str.len().sum()

198469

In [ ]:
con = sqlite3.connect("/mnt/e/Desktop/1/b本地化/Nova文字表/nova_texts.sqlite")
SQL="""
select 
file_name, 
sheet_name, 
text_id, 
key,
en, 
sc
from nova_texts
where en like '%ffairs%'
"""
df = pd.read_sql_query(SQL,con)
con.close()
df.head()